# GPT

## Einleitung

### API

Unten befindet sich eine fertige Umsetzung einer API, mit welcher wir GPT verwenden können.

In [ ]:
import openai
openai.api_key = ...

class OpenaiClient:
    def get_gpt_completion(self, prompt: str, **kwargs) -> str:
        completion = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[{
                'role': 'user',
                'content': prompt
            }],
            temperature=0.25,
            max_tokens=512,
            **kwargs
        )
        completion = completion['choices'][0]['message']['content'].encode("utf-8", "replace").decode()

        return completion

    def get_gpt_embedding(self, text: str, **kwargs):
        response = openai.Embedding.create(
            input=text,
            model="text-embedding-ada-002",
            **kwargs
        )
        embedding = response['data'][0]['embedding']
        return embedding

Wir können die API nun wie folgt verwenden:

In [ ]:
client = OpenaiClient()

result = client.get_gpt_completion('Hallo, wer bist du?')
print(result)

### Embeddings

Neben Vervollständigung von Text bietet OpanAI auch Embeddings an. Embeddings sind eine Liste von Zahlen, welche die Bedeutung eines Texts repräsentieren. Führe den Code unten aus um die Ähnlichkeit verschiedener Texte zu ermitteln. Du kannst die Texte gerne auch selber anpassen. Machen die Resultate Sinn für dich?

In [ ]:
from numpy import dot
from numpy.linalg import norm

TEXTS = [
    'Ich gehe gerne zur Schule, weil ich dort so viel lernen kann.',
    'In der ersten Klasse wurde mir so viel beigebracht',
    'A Random Walk Down Wall Street',
]

client = OpenaiClient()
text_embeddings = [client.get_gpt_embedding(t) for t in TEXTS]

for i, text_1 in enumerate(TEXTS):
    for j, text_2 in enumerate(TEXTS):
        if j > i:
            continue

        emedding_1 = text_embeddings[i]
        emedding_2 = text_embeddings[j]

        similarity = dot(emedding_1, emedding_2)/(norm(emedding_1)*norm(emedding_2))

        print(f'TEXT 1: {text_1}')
        print(f'TEXT 2: {text_2}')
        print(f'ÄHNLICHKEIT: {similarity}')
        print()
        

### Chatbot

Unten findest du eine Implementierung eines einfachen Chatbots. Verstehst du die Funktionsweise?

In [ ]:
import numpy as np

PROMPT = lambda query, hint: f"""
    Du bist ein Chatbot, der Leute zum lachen bringen soll. Gib dem User eine lustige Antwort.
    Verwende den Hinweis, falls sinnvoll.

    HINWEIS: {hint}
    USER: {query}
    ANTWORT: """

KNOWLEDGE_BASE = {
    'Ich bin traurig': 'Ein Witz für traurige Leute: "Alles hat ein Ende, nur die Wurst hat zwei."',
    'Ich weiss nicht was ich tun soll': 'Ein Witz für unentschlossene: "I used to think I was indecisive. But now I’m not so sure."'
}

class Chatbot:
    def __init__(self, knowledge_base=KNOWLEDGE_BASE):
        self.client = OpenaiClient()
        self.knowledge_base = KNOWLEDGE_BASE
        self.knowledge_base_embeddings = [self.client.get_gpt_embedding(t) for t in self.knowledge_base.keys()]

    def respond(self, request):
        request_embedding = self.client.get_gpt_embedding(request)

        similarities = [dot(request_embedding, e)/(norm(request_embedding)*norm(e)) for e in self.knowledge_base_embeddings]
        most_similar_index = np.argmax(similarities)

        query = PROMPT(request, list(self.knowledge_base.values())[most_similar_index])
        response = self.client.get_gpt_completion(query)

        print(response)
        
        
chatbot = Chatbot()

In [ ]:
chatbot.respond('Ich weiss einfach nicht mehr weiter')